<a href="https://colab.research.google.com/github/KU-BIG/KUBIG_2022_FALL/blob/main/1.%20%EB%B0%A9%ED%95%99%EB%B6%84%EB%B0%98/%EB%94%A5%EB%9F%AC%EB%8B%9D/%EA%B3%BC%EC%A0%9C/6%EC%A3%BC%EC%B0%A8/6%E1%84%8C%E1%85%AE%E1%84%8E%E1%85%A1_%E1%84%80%E1%85%AA%E1%84%8C%E1%85%A6_%EA%B3%B5%EB%8F%84%EC%9B%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

## Task1

빈 칸을 채워주세요!

단계별 output이 github 파일에는 남아있으니 그 output과 동일한 형태인지 확인하면서 진행해주시면 됩니다~

In [4]:
#1. 생성할 문장 데이터

sentence = ("Brick walls are there for a reason and you must not think "
            "that the brick walls aren't there to keep us out, but rather "
            "in this way that the brick walls are there to show us how badly we want things.")

In [20]:
#2. 문자 집합 만들기
world_set = list(set(sentence))

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {value : i for i,value in enumerate(world_set)}

In [21]:
print(vocab)

{'g': 0, 'e': 1, 'f': 2, 'm': 3, 'u': 4, 'B': 5, 'l': 6, 'w': 7, 'd': 8, 'p': 9, 'a': 10, 's': 11, '.': 12, ',': 13, 'h': 14, 'r': 15, 'o': 16, 'b': 17, 'k': 18, 'i': 19, 't': 20, ' ': 21, 'y': 22, 'n': 23, "'": 24, 'c': 25}


In [22]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 26


In [23]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 10  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

In [24]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+sequence_length]
  y_str = sentence[i+1:i+1+sequence_length]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 Brick wall -> rick walls
1 rick walls -> ick walls 
2 ick walls  -> ck walls a
3 ck walls a -> k walls ar
4 k walls ar ->  walls are
5  walls are -> walls are 
6 walls are  -> alls are t
7 alls are t -> lls are th
8 lls are th -> ls are the
9 ls are the -> s are ther
10 s are ther ->  are there
11  are there -> are there 
12 are there  -> re there f
13 re there f -> e there fo
14 e there fo ->  there for
15  there for -> there for 
16 there for  -> here for a
17 here for a -> ere for a 
18 ere for a  -> re for a r
19 re for a r -> e for a re
20 e for a re ->  for a rea
21  for a rea -> for a reas
22 for a reas -> or a reaso
23 or a reaso -> r a reason
24 r a reason ->  a reason 
25  a reason  -> a reason a
26 a reason a ->  reason an
27  reason an -> reason and
28 reason and -> eason and 
29 eason and  -> ason and y
30 ason and y -> son and yo
31 son and yo -> on and you
32 on and you -> n and you 
33 n and you  ->  and you m
34  and you m -> and you mu
35 and you mu -> nd you mus
36

In [25]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[5, 15, 19, 25, 18, 21, 7, 10, 6, 6]
[15, 19, 25, 18, 21, 7, 10, 6, 6, 11]


In [26]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [np.eye(len(vocab))[x] for x in x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [27]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([188, 10, 26])
레이블의 크기 : torch.Size([188, 10])


In [28]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0.,

In [29]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([15, 19, 25, 18, 21,  7, 10,  6,  6, 11])


In [108]:
X.size()

torch.Size([188, 10, 26])

In [117]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()

    self.input_dim = input_dim
    self.hidden_dim = hidden_dim
    self.layers = layers

    self.rnn = torch.nn.RNN(input_dim, hidden_dim,layers,batch_first = True)
    self.fc = torch.nn.Linear(hidden_dim ,vocab_size)
  
  def forward(self, x):
    h0 = torch.zeros(self.layers,X.size()[0],self.hidden_dim) #(2,188,26)

    # Propagate input through RNN
    # Input: (batch, seq_len, input_size) => X.size() = (188,10,26)
    # hidden: (num_layers * num_directions, batch, hidden_size) =>rnn(X)[1].size() = (2,188,26)

    out, _ = self.rnn(x,h0)
    out = self.fc(out)
  
    return out

In [133]:
X.size()

torch.Size([188, 10, 26])

In [134]:
rnn = torch.nn.RNN(26, 26,2,batch_first = True)

In [138]:
rnn(X)[0].size()

torch.Size([188, 10, 26])

In [139]:
rnn(X)[1].size()

torch.Size([2, 188, 26])

In [132]:
torch.zeros(2,X.size()[0],26).size()

torch.Size([2, 188, 26])

In [118]:
net = Net(vocab_size, hidden_size, 2) #(26,26,2)

In [119]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [120]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([188, 10, 26])


In [123]:
outputs.view(-1,26).shape

torch.Size([1880, 26])

In [122]:
Y.view(-1).shape

torch.Size([1880])

In [127]:
outputs.view(-1,26)

tensor([[-3.7998, -1.0167, -2.1204,  ...,  0.3425, -4.2237, -2.6636],
        [ 0.6412,  4.8120,  2.1728,  ..., -1.0013,  0.6091, -0.8956],
        [ 1.0872,  2.5565, -3.6666,  ...,  4.1055, -1.0850,  8.7898],
        ...,
        [ 5.8168,  4.2398, -0.9342,  ..., -2.5350,  4.1850, -0.0251],
        [ 0.0806,  2.6463, -3.5842,  ..., -0.6897,  0.0505, -0.7754],
        [ 0.5870, -0.9869,  3.0740,  ...,  1.2036,  2.7333, -1.8418]],
       grad_fn=<ViewBackward0>)

In [128]:
Y.view(-1)

tensor([15, 19, 25,  ...,  0, 11, 12])

In [124]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    #output의 shape : 188,10,26
    #label의 shape : 188,10
    loss = criterion(outputs.view(-1,26), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

euiiieeieiiiiuiiiyiyiiiyiiiuiiiiiiuiieieiiiiiiiuiyiuiyiiieiiiiuiyiuiiiiieeieiiiiuiieiuiyiyiiiyuieiiiiiiiiiiuiiiiiiiyiyiiiiiyiiieieuiyiiuiyiuiiiiieeieiiiiuiiiyiyiiiyuiyiiiiiiiiiiiiiuiuieuiiiiiiyiiii
ete   e          e    e     e       e  e   e    e    e     s    e   ee   e        e    e                  e     e  e e e   er  e    e    e   ee   e          e        e        f       e  e     e    
                                                                                                                                                                                                     
                                                                                                                                                                                                     
                                                                                                                                                                                                     
          

In [125]:
predict_str

"rick walls are there tor a reason and you must not think that the brick walls are 't there to seep us out, but rather in this way that the brick walls are there to show us how badly we want thinksy"

In [126]:
sentence

"Brick walls are there for a reason and you must not think that the brick walls aren't there to keep us out, but rather in this way that the brick walls are there to show us how badly we want things."

결과가 어떤가요?? 마지막 에폭의 문장이 그럴싸한가요?

## Task2

위 sentence는 제가 임의로 생성한 문장들입니다.

마음에 드시는 문구 가져오셔서 문장이 어떻게 생성되는지 확인해보세요! 

영어가 아닌 한국어로 시도해보는 것도 좋겠죠? 

수정이 많이 필요(토큰화 등) 할 수 있으나 한번 시도해보시는 것 권장드립니다 :)

위 베이스라인은 어디든 수정하셔도 좋고 조금 더 자연스러운 문장이 나올 수 있게 다양한 시도를 해보세요!

조건 : 문장 3개 이상, 연결성이 있는 문장을 " " 으로 구분하여 ( )에 넣기

In [154]:
#!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
#%cd Mecab-ko-for-Google-Colab
#!bash install_mecab-ko_on_colab190912.sh

In [324]:
#1. 생성할 문장 데이터
text = ("우리 매번 같은 시간에 있다는 거 "
        "시간을 보면 네가 내 안에 그려져 "
        "같이 있어도 너만 생각할게")

In [325]:
#2. 문자 집합 만들기
from konlpy.tag import Mecab
tokenizer = Mecab()
word = tokenizer.morphs(text)
word_set =list(set(word))


## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {value : i for i,value in enumerate(word_set)}

In [326]:
print(vocab)

{'가': 0, '시간': 1, '생각': 2, '네': 3, '너': 4, '같': 5, '할': 6, '다는': 7, '은': 8, '을': 9, '보': 10, '어도': 11, '만': 12, '내': 13, '에': 14, '거': 15, '게': 16, '있': 17, '매번': 18, '면': 19, '우리': 20, '같이': 21, '그려져': 22, '안': 23}


In [327]:
print(word)

['우리', '매번', '같', '은', '시간', '에', '있', '다는', '거', '시간', '을', '보', '면', '네', '가', '내', '안', '에', '그려져', '같이', '있', '어도', '너', '만', '생각', '할', '게']


In [328]:
#3. 형태소 집합 크기 확인

vocab_size = len(vocab)
print('형태소 집합 크기 : {}'.format(vocab_size))

형태소 집합 크기 : 24


In [329]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 8  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.007

In [330]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(word) - sequence_length):
  x_str = word[i:i+sequence_length]
  y_str = word[i+1:i+1+sequence_length]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 ['우리', '매번', '같', '은', '시간', '에', '있', '다는'] -> ['매번', '같', '은', '시간', '에', '있', '다는', '거']
1 ['매번', '같', '은', '시간', '에', '있', '다는', '거'] -> ['같', '은', '시간', '에', '있', '다는', '거', '시간']
2 ['같', '은', '시간', '에', '있', '다는', '거', '시간'] -> ['은', '시간', '에', '있', '다는', '거', '시간', '을']
3 ['은', '시간', '에', '있', '다는', '거', '시간', '을'] -> ['시간', '에', '있', '다는', '거', '시간', '을', '보']
4 ['시간', '에', '있', '다는', '거', '시간', '을', '보'] -> ['에', '있', '다는', '거', '시간', '을', '보', '면']
5 ['에', '있', '다는', '거', '시간', '을', '보', '면'] -> ['있', '다는', '거', '시간', '을', '보', '면', '네']
6 ['있', '다는', '거', '시간', '을', '보', '면', '네'] -> ['다는', '거', '시간', '을', '보', '면', '네', '가']
7 ['다는', '거', '시간', '을', '보', '면', '네', '가'] -> ['거', '시간', '을', '보', '면', '네', '가', '내']
8 ['거', '시간', '을', '보', '면', '네', '가', '내'] -> ['시간', '을', '보', '면', '네', '가', '내', '안']
9 ['시간', '을', '보', '면', '네', '가', '내', '안'] -> ['을', '보', '면', '네', '가', '내', '안', '에']
10 ['을', '보', '면', '네', '가', '내', '안', '에'] -> ['보', '면', '네', '가', '내', '안', '에', '그려

In [331]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[20, 18, 5, 8, 1, 14, 17, 7]
[18, 5, 8, 1, 14, 17, 7, 15]


In [332]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [np.eye(len(vocab))[x] for x in x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [333]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([19, 8, 24])
레이블의 크기 : torch.Size([19, 8])


In [334]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()

    self.input_dim = input_dim
    self.hidden_dim = hidden_dim
    self.layers = layers

    self.rnn = torch.nn.RNN(input_dim, hidden_dim,layers,batch_first = True)
    self.fc = torch.nn.Linear(hidden_dim ,vocab_size)
  
  def forward(self, x):
    h0 = torch.zeros(self.layers,X.size()[0],self.hidden_dim) #(2,26,17)

    # Propagate input through RNN
    # Input: (batch, seq_len, input_size) => X.size() = (26,3,17)
    # hidden: (num_layers * num_directions, batch, hidden_size) =>rnn(X)[1].size() = (2,26,17)

    out, _ = self.rnn(x,h0)
    out = self.fc(out)
  
    return out

In [335]:
net = Net(vocab_size, hidden_size, 2) #(17,17,2)

##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([19, 8, 24])


In [336]:
outputs.size()

torch.Size([19, 8, 24])

In [337]:
Y.size()

torch.Size([19, 8])

In [346]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1,24), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = "우리"
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([word_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += word_set[result[-1]]

    print(predict_str)

우리매번같은시간에있다는거시간을보면네가내안에그려져같이있어도너만생각할게
우리매번같은시간에있다는거시간을보면네가내안에그려져같이있어도너만생각할게
우리매번같은시간에있다는거시간을보면네가내안에그려져같이있어도너만생각할게
우리매번같은시간에있다는거시간을보면네가내안에그려져같이있어도너만생각할게
우리매번같은시간에있다는거시간을보면네가내안에그려져같이있어도너만생각할게
우리매번같은시간에있다는거시간을보면네가내안에그려져같이있어도너만생각할게
우리매번같은시간에있다는거시간을보면네가내안에그려져같이있어도너만생각할게
우리매번같은시간에있다는거시간을보면네가내안에그려져같이있어도너만생각할게
우리매번같은시간에있다는거시간을보면네가내안에그려져같이있어도너만생각할게
우리매번같은시간에있다는거시간을보면네가내안에그려져같이있어도너만생각할게
우리매번같은시간에있다는거시간을보면네가내안에그려져같이있어도너만생각할게
우리매번같은시간에있다는거시간을보면네가내안에그려져같이있어도너만생각할게
우리매번같은시간에있다는거시간을보면네가내안에그려져같이있어도너만생각할게
우리매번같은시간에있다는거시간을보면네가내안에그려져같이있어도너만생각할게
우리매번같은시간에있다는거시간을보면네가내안에그려져같이있어도너만생각할게
우리매번같은시간에있다는거시간을보면네가내안에그려져같이있어도너만생각할게
우리매번같은시간에있다는거시간을보면네가내안에그려져같이있어도너만생각할게
우리매번같은시간에있다는거시간을보면네가내안에그려져같이있어도너만생각할게
우리매번같은시간에있다는거시간을보면네가내안에그려져같이있어도너만생각할게
우리매번같은시간에있다는거시간을보면네가내안에그려져같이있어도너만생각할게
우리매번같은시간에있다는거시간을보면네가내안에그려져같이있어도너만생각할게
우리매번같은시간에있다는거시간을보면네가내안에그려져같이있어도너만생각할게
우리매번같은시간에있다는거시간을보면네가내안에그려져같이있어도너만생각할게
우리매번같은시간에있다는거시간을보면네가내안에그려져같이있어도너만생각할게
우리매번같은시간에있다는거시간을보면네가내안에그려져같이있어도너만생각할게
우리매번같은시간에있다는거시간을보면네가내안에그려져같이있어도너만생각할게
우리매번같은시간에있다는

In [347]:
predict_str

'우리매번같은시간에있다는거시간을보면네가내안에그려져같이있어도너만생각할게'

In [348]:
text

'우리 매번 같은 시간에 있다는 거 시간을 보면 네가 내 안에 그려져 같이 있어도 너만 생각할게'